In [6]:
%matplotlib qt

In [1]:
from sklearn.tree import DecisionTreeClassifier as Tree
from sklearn.ensemble import RandomForestClassifier as Forest
from sklearn.ensemble import ExtraTreesClassifier as EForest
from sklearn.cross_validation import train_test_split as sk_split
from sklearn.neighbors import KNeighborsClassifier as KNN
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import sys

def Rotate(coords,theta):
    out = np.zeros(coords.shape)
    out[:,0] = coords[:,0]*np.cos(theta) + coords[:,1]*np.sin(theta)
    out[:,1] = -coords[:,0]*np.sin(theta) + coords[:,1]*np.cos(theta)
    return out

def permutation_importance(tree,test_data,test_target): # estimate variable importance using test data
    is_verbose = tree.get_params()['verbose']
    tree.set_params(verbose=False)
    importances = np.zeros(test_data.shape[1])
    original_score = tree.score(test_data,test_target)
    for i in xrange(test_data.shape[1]): # scramble each column and get % increase in error rate (Breinman importance)
        local = test_data.copy()
        np.random.shuffle(local[:,i])
        importances[i] = (original_score - tree.score(local,test_target))/(1-original_score)
        if is_verbose:
            sys.stdout.write('.')
            
    tree.set_params(verbose=is_verbose)
    return importances

In [2]:
# Load data
train_data = pd.read_csv('train.csv')
train_labels = pd.read_csv('train_label.csv')
compete_data = pd.read_csv('test.csv')
compete_id = compete_data.id
N = train_data.shape[0]

target = np.zeros(N,dtype=np.int)
target[np.array(train_labels['status_group']=='non functional')] = 0
target[np.array(train_labels['status_group']=='functional needs repair')] = 1
target[np.array(train_labels['status_group']=='functional')] = 2

# Manual data processing

# Convert date recorded to days since the first recording
s = train_data['date_recorded']
sc = compete_data['date_recorded']
s = s.apply(lambda date_string: np.datetime64(date_string))
sc = sc.apply(lambda date_string: np.datetime64(date_string))
min_date = s.min()
min_datec = sc.min()
s = (s-s.min())/np.timedelta64(1,'D')
sc = (sc-sc.min())/np.timedelta64(1,'D')
train_data['date_recorded']=s
compete_data['date_recorded']=sc

train_data['region_code'] = 'r'+train_data['region_code'].astype(np.str) # Regions are categorical
compete_data['region_code'] = 'r'+compete_data['region_code'].astype(np.str)

#                uniform       copy of quantity      unique
for feature in ['recorded_by','quantity_group','id','num_private']:
    train_data.drop(feature, axis=1, inplace=True) # uniform
    compete_data.drop(feature,axis=1,inplace=True)
####### EMPRICAL GUESSES
## General features that will correlate with higher-detail versions of that feature, e.g. source_type generalized source
for feature in ['extraction_type_group','extraction_type_class','payment_type',
                'quality_group','source_type','source_class','waterpoint_type_group',
                'scheme_management','scheme_name','date_recorded','management_group','basin']:
    train_data.drop(feature,axis=1,inplace=True)
    compete_data.drop(feature,axis=1,inplace=True)
    
# Region proxy variables
for feature in ['region','region_code','district_code','lga','wpt_name','ward','subvillage']:
    train_data.drop(feature,axis=1,inplace=True)
    compete_data.drop(feature,axis=1,inplace=True)
    
# After removing all of the above, the following has mean permutation importances
# of <=0.0003 each (as assessed over 15 samples of 50 trees each)
#######

train_data.fillna("was_nan",inplace=True)
compete_data.fillna("was_nan",inplace=True)
# Random data to help assess variable importance
#train_data['random_1'] = np.random.uniform(0.0,1.0,N)
#train_data['random_2'] = np.random.uniform(0.0,1.0,N)
#train_data['random_3'] = np.random.binomial(1,0.5,N)
#train_data['random_4'] = np.random.binomial(1,0.1,N)


# For all categorical data: create binary columns for each category that represents at least p% of the data
p = 0.01
feature_factors = {}
for f in list(train_data.columns):
    # Only modify string data
    if train_data[f].dtype == np.object:
        sizes = train_data.groupby(f).size()/N
        sizes.sort(ascending=False)
        sizes = sizes[sizes>p]
#        print sizes[sizes>p]
#        print ""
#        print f
        appended = np.zeros(N,dtype=np.bool)
        # The list of categories with at least p% of the training data for feature f
        salient_categories = list(sizes.keys())
        feature_factors[f] = salient_categories
        for cat in salient_categories:
            # Append the binary column
            train_data[f+'__'+str(cat)] = (train_data[f]==cat)
#            print "\t", cat
        train_data.drop(f,axis=1,inplace=True)
    # Done!
    
### Perform the same one-hot encoding on the competition data
for f in list(compete_data.columns):
    # Only modify string (categorical) data
    if compete_data[f].dtype == np.object:
        for cat in feature_factors[f]:
            # Create binary column for trained factors for this feature
            compete_data[f+'__'+str(cat)] = (compete_data[f]==cat)
        # remove the original column
        compete_data.drop(f,axis=1,inplace=True)

## After removing all of the above, the following has mean permutation importances
## of <=0.0003 each (as assessed over 10 samples of 50 trees each)
#for feature in ['funder__Rwssp', 'funder__District Council','funder__0', 'funder__Germany Republi', 'funder__Tcrs',
#       'installer__Commu', 'installer__DANIDA', 'installer__0', 'installer__TCRS', 'installer__Central government',
#       'installer__CES', 'public_meeting__was_nan', 'permit__was_nan', 'extraction_type__swn 80', 
#       'extraction_type__afridev', 'extraction_type__ksb', 'management__parastatal',
#       'payment__unknown', 'water_quality__soft', 'water_quality__milky', 'quantity__unknown']:
#for feature in ['funder__Unicef','funder__0','funder__Kkkt','permit__was_nan',
#                'management__water board','water_quality__salty','quantity__unknown',
#                'source__lake']:
#    train_data.drop(feature,axis=1,inplace=True)
#    compete_data.drop(feature,axis=1,inplace=True)
    

In [3]:
XY = np.array([train_data.longitude,train_data.latitude]).T
XYc = np.array([compete_data.longitude,compete_data.latitude]).T
for coordinate in ['latitude','longitude']:
    train_data.drop(coordinate,axis=1,inplace=True)
    compete_data.drop(coordinate,axis=1,inplace=True)

r = 5 # total number of rotations (including original coordinates)
for i,theta in enumerate(np.linspace(0,np.pi/4,r)):
    coords = Rotate(XY, theta)
    train_data['longitude_r%d'%(i)] = coords[:,0]
    train_data['latitude_r%d'%(i)] = coords[:,1]
    coordsc = Rotate(XYc, theta)
    compete_data['longitude_r%d'%(i)] = coordsc[:,0]
    compete_data['latitude_r%d'%(i)] = coordsc[:,1]

data_matrix = train_data.as_matrix().astype(np.float);
data_matrix_compete = compete_data.as_matrix().astype(np.float);
print data_matrix.shape
print data_matrix_compete.shape
    
# These functions return an array with several columns and a number of
# rows equal to the leading dimension of distances and classes.
def inverse_distance_class(distances,neighbors,classes):
    d = distances.copy()
    d[d==0] = np.inf
    d = 1.0/d
    class_sums = np.zeros((distances.shape[0],3))
    for i in range(distances.shape[0]):
        for c in range(3):
            class_sums[i,c] = np.sum(dists[i,classes[neighbors[i]]==c])
    return class_sums

def sigmoid_distance_class(distances,neighbors,classes):
    class_sums = np.zeros((distances.shape[0],3))
    mu = 0.001
    kt = 0.001
    sig = distances/(1+np.exp((distances-mu)/kt))
    for i in range(neighbors.shape[0]):
        for c in range(3):
            class_sums[i,c] = np.sum(sig[i,classes[neighbors[i]]==c])
    return class_sums

def long_sigmoid_distance_class(distances,neighbors,classes):
    class_sums = np.zeros((distances.shape[0],3))
    mu = 0.001
    kt = 0.007
    sig = distances/(1+np.exp((distances-mu)/kt))
    for i in range(neighbors.shape[0]):
        for c in range(3):
            class_sums[i,c] = np.sum(sig[i,classes[neighbors[i]]==c])
    return class_sums

def num_neighbors_class(distances,neighbors,classes):
    class_sums = np.zeros((neighbors.shape[0],3))
    for i in range(neighbors.shape[0]):
        for c in range(3):
            class_sums[i,c] = np.sum(classes[neighbors[i]]==c)
    return class_sums

generators = [inverse_distance_class, sigmoid_distance_class,
              long_sigmoid_distance_class,num_neighbors_class]
#generators = []

# Include kNN information using the tuned k=30 results from the initial investigation using kNN classifiers
# This gives the forest access to (unnormalized) kNN information
knn = KNN(n_neighbors=50)
knn.fit(XY,target)

dists,neighbors = knn.kneighbors(return_distance=True)
for generator in generators:
    location_data = generator(dists,neighbors,target)
    print data_matrix.shape
    print location_data.shape
    data_matrix = np.c_[data_matrix,location_data]

dists,neighbors = knn.kneighbors(XYc,return_distance=True)
for generator in generators:
    location_data = generator(dists,neighbors,target)
    data_matrix_compete = np.c_[data_matrix_compete,location_data]

(59400, 97)
(14850, 97)
(59400, 97)
(59400, 3)
(59400, 100)
(59400, 3)
(59400, 103)
(59400, 3)
(59400, 106)
(59400, 3)


In [374]:
plt.figure()

In [4]:
data_train, data_test, target_train, target_test = sk_split(data_matrix,target,test_size=0.1)

In [5]:
# Plot accuracy vs number of trees
accuracy = []
n_est = []
forest = Forest(n_estimators=1, criterion='gini', n_jobs=4,verbose=False,max_features=8,bootstrap=True,oob_score=False,
                warm_start=True)
forest.fit(data_train,target_train)
accuracy += [forest.score(data_test,target_test)]
n_est += [forest.n_estimators]
for i in range(1,5):
    forest.set_params(n_estimators=i*20)
    forest.fit(data_train,target_train)
    accuracy += [forest.score(data_test,target_test)]
    n_est += [forest.n_estimators]
    print accuracy[-1:]
plt.plot(n_est,accuracy,'-^')

[0.82205387205387204]
[0.82727272727272727]
[0.83063973063973062]
[0.82946127946127945]


In [7]:
plt.plot(n_est,accuracy,'--s')

In [430]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier as knn_classifier

In [54]:
T = Tree(criterion='gini',splitter='best',max_depth=2,min_samples_split=2,min_samples_leaf=1,max_features=None)
T.fit(data_matrix,target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [55]:
v = np.sum(T.tree_.value,axis=0).squeeze()
print v
print T.tree_.value

[ 22824.   4317.  32259.]
[[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]]

 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00]]

 [[  1.06780000e+04   2.41700000e+03   9.31700000e+03]]

 [[  6.09400000e+03   1.86300000e+03   2.27850000e+04]]

 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00]]

 [[  5.61500000e+03   3.30000000e+01   1.02000000e+02]]

 [[  4.37000000e+02   4.00000000e+00   5.50000000e+01]]]


In [73]:
plt.figure()
plt.pie(v,colors=['red','yellow','blue']); plt.gca().set_aspect('equal')

In [74]:
plt.legend(['non functional','functional needs repair','functional'])

In [69]:
print T.tree_.feature
print T.tree_.threshold
print T.tree_.children_left
print T.tree_.children_right
print T.tree_.impurity
print
for i in range(7):
    print T.tree_.value[i].squeeze()

[ 71 108  -2  -2  41  -2  -2]
[  0.5  26.5  -2.   -2.    0.5  -2.   -2. ]
[ 1  2 -1 -1  5 -1 -1]
[ 4  3 -1 -1  6 -1 -1]
[ 0.55213908  0.52920599  0.58855467  0.40770155  0.06048813  0.04605768
  0.21139275]

[ 0.  0.  0.]
[ 0.  0.  0.]
[ 10678.   2417.   9317.]
[  6094.   1863.  22785.]
[ 0.  0.  0.]
[ 5615.    33.   102.]
[ 437.    4.   55.]


In [57]:
list(train_data.columns)[41]

'extraction_type__nira/tanira'

In [36]:
plt.figure()
plt.subplot(1,2,1)
plt.pie(T.tree_.value[1].squeeze(),colors=['red','yellow','blue']); plt.gca().set_aspect('equal')
plt.subplot(1,2,2)
plt.pie(T.tree_.value[2].squeeze(),colors=['red','yellow','blue']); plt.gca().set_aspect('equal')

In [72]:
plt.figure()
for p,index in enumerate([2,3,5,6]):
    plt.subplot(1,4,p+1)
    plt.pie(T.tree_.value[index].squeeze(),colors=['red','yellow','blue']); plt.gca().set_aspect('equal')

In [431]:
train_data.shape

(59400, 87)

In [435]:
forest = EForest(n_estimators=200, criterion='gini', n_jobs=4,verbose=True,max_features='auto',bootstrap=True,
                warm_start=False,min_samples_split=2,min_samples_leaf=1,oob_score=True)#,
#                class_weight={0:0.5,1:0.5,2:1})
forest.fit(data_matrix,target)
forest.oob_score_

[Parallel(n_jobs=4)]: Done   1 out of 200 | elapsed:    0.8s remaining:  2.6min
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   10.5s finished


0.83240740740740737

In [ ]:
for i in range(3):
    p = np.argmax(forest.oob_decision_function_,axis=1)
    print i, " ", np.sum((target==i)&(p==i))/np.sum(target==i,dtype=np.float)*100.0

In [91]:
r = 1
dists,neighbors = knn.kneighbors(XY,return_distance=True)
gps_only = np.zeros((neighbors.shape[0],2*r))
for i,theta in enumerate(np.linspace(0,np.pi/4,r)):
    print "Rotation %d: %f" %(i,theta)
    gps_only[:,i*2:(i+1)*2] = Rotate(XY,theta)
#for generator in generators:
#    gps_only = np.c_[gps_only,generator(dists,neighbors,target)]

Rotation 0: 0.000000


In [92]:
forest = Forest(n_estimators=1, criterion='gini',n_jobs=4,verbose=True,
                max_features=1, bootstrap=True, oob_score=True,
                min_samples_split=2, min_samples_leaf=1)
forest.fit(gps_only,target)
forest.oob_score_

[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    1.4s finished
/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


0.46870370370370368

In [93]:
for i in range(3):
    p = np.argmax(forest.oob_decision_function_,axis=1)
    print i, " ", np.sum((target==i)&(p==i))/np.sum(target==i,dtype=np.float)*100.0

0   84.1482649842
1   9.75214269168
2   25.4626615828


In [95]:
######################### OVERLAY EACH INDIVIDUAL TREE FROM THE FOREST
plt.figure()
Ntiles = 8 * 10**4
xmin = 33.15; xmax = 33.40
ymin = -9.03; ymax = -8.81
s = np.sqrt((xmax-xmin)*(ymax-ymin)/Ntiles)
nx = np.int((xmax-xmin)/s); ny = np.int((ymax-ymin)/s)
print nx, " ", ny

xx,yy = np.meshgrid(np.linspace(xmin, xmax, nx),
                    np.linspace(ymin, ymax, ny))
cmap = plt.cm.RdYlGn
model = forest
estimator_alpha = 1.0 / len(model.estimators_)
xy = np.c_[xx.ravel(), yy.ravel()]
coords = np.zeros((xy.shape[0],2*r))
for i,theta in enumerate(np.linspace(0,np.pi/4,r)):
    coords[:,2*i:2*i+2] = Rotate(xy,theta)
dists,neighbors = knn.kneighbors(xy,return_distance=True)
#for generator in generators:
#    coords = np.c_[coords,generator(dists,neighbors,target)]
for tree in model.estimators_:
    Z = tree.predict(coords)
    Z = Z.reshape(xx.shape)
    cs = plt.gca().pcolorfast(np.linspace(xmin,xmax),np.linspace(ymin,ymax),
                              Z,cmap=cmap,alpha=estimator_alpha)
#    cs = plt.pcolormesh(xx, yy, Z, alpha=estimator_alpha, cmap=cmap,
#                        shading='face',edgecolor='None')
#    cs.set_linewidths(0)
#    cs.set_linestyle('--')
#Z = forest.predict(coords)
#Z = Z.reshape(xx.shape)
#cs = plt.gca().pcolorfast(np.linspace(xmin,xmax),np.linspace(ymin,ymax),
#                          Z,cmap=cmap,alpha=0.5)
#cs.set_linewidths(0)
#cs.set_linestyle('--')


plt.gca().set_xlim([xmin,xmax])
plt.gca().set_ylim([ymin,ymax])
plt.gca().set_aspect('equal')
plt.plot(XY[target==0,0],XY[target==0,1],'ro')
plt.plot(XY[target==1,0],XY[target==1,1],'yo')
plt.plot(XY[target==2,0],XY[target==2,1],'go')
plt.show()

301   265


In [174]:
predictions = forest.predict(data_matrix_compete)

[Parallel(n_jobs=4)]: Done   1 out of  81 | elapsed:    0.0s remaining:    3.0s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    1.4s finished


In [96]:
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
plt.gcf().tight_layout()

In [ ]:
plt.savefig()

In [175]:
predictions_for_export = np.zeros(predictions.shape,dtype=np.object)
predictions_for_export[predictions==0] = 'non functional'
predictions_for_export[predictions==1] = 'functional needs repair'
predictions_for_export[predictions==2] = 'functional'
predictions_for_export = np.array([compete_id, predictions_for_export]).T

In [176]:
predictions_for_export.shape

(14850, 2)

In [177]:
predictions_for_export

array([[50785, 'non functional'],
       [51630, 'functional'],
       [17168, 'functional'],
       ..., 
       [28749, 'non functional'],
       [33492, 'functional'],
       [68707, 'non functional']], dtype=object)

In [178]:
np.savetxt("submit12.csv",predictions_for_export,fmt='%s',delimiter=',',header='id,status_group')
##### REMEMBER: EDIT CSV FILE TO REMOVE HEADER'S LEADING # AND SPACE

In [206]:
r

3

In [181]:
plt.figure(); plt.plot(XY[:,0],XY[:,1],'ko',markerfacecolor=None);
plt.gca().set_aspect('equal')
plt.plot(XYc[:,0],XYc[:,1],'ro',markerfacecolor=None)

In [6]:
#### TREE PLOTTING
trees = forest.estimators_
t = trees[0]

In [7]:
t_ = t.tree_

In [461]:
t_.n_node_samples

array([37398, 21410, 19189, ...,     1,     7,    26])

In [462]:
depth = np.zeros(t_.value.shape)

In [481]:
np.power(0.9,np.log2(t_.children_left.shape,))

array([ 0.22219984])

In [474]:
plt.plot(t_.children_left[t_.children_left>0])

In [475]:
plt.plot(t_.children_right[t_.children_right>0])

In [8]:
from matplotlib import collections  as mc

In [70]:
def draw_tree(tree,scale = 0.8, angle = np.pi/4):
    t_ = tree.tree_
    num_nodes = t_.value.shape[0]
    left_shift = np.array([-np.sin(angle),-np.cos(angle)])
    right_shift = np.array([np.sin(angle),-np.cos(angle)])
    levels = np.zeros(num_nodes,dtype=np.int)
    locations = np.zeros((num_nodes,2))
    segments = []
    for node in range(locations.shape[0]):
        left = t_.children_left[node]
        right = t_.children_right[node]
        if left > 0:
            levels[[left,right]] = levels[node]+1
            reduction = t_.impurity[node]*t_.n_node_samples[node] - t_.impurity[left]*t_.n_node_samples[left] - t_.impurity[right]*t_.n_node_samples[right]
            locations[left] = locations[node] + t_.n_node_samples[left]*left_shift
            locations[right] = locations[node] + t_.n_node_samples[right]*right_shift
#            locations[left] = locations[node] + reduction*left_shift
#            locations[right] = locations[node] + reduction*right_shift
            segments += [[locations[node],locations[left]]]
            segments += [[locations[node],locations[right]]]
#    plt.figure()
#    lc = mc.LineCollection(segments)
#    plt.gca().add_collection(lc)
    return segments

In [72]:
segments = draw_tree(trees[3],scale=0.5, angle = np.pi/2.5)
ax.add_collection(mc.LineCollection(np.array(segments),colors='black'))

In [71]:
ax=plt.gca()

In [26]:
lc = mc.LineCollection(np.array(segments))

In [27]:
plt.gca().add_collection(lc)

In [28]:
plt.show()

In [77]:
plt.tight_layout()

In [74]:
plt.tight_layout()

In [75]:
ax.set_xticks([])

[]

In [76]:
ax.set_yticks([])

[]

In [89]:
from sklearn import tree

In [90]:
tree.export_graphviz(trees[3],out_file='tree.dot',feature_names=['lon0','lat0','lon1','lat1','lon2','lat2','inv_n','inv_r','inv_f','sig_n','sig_r','sig_f','Lsig_n','Lsig_r','Lsig_f','N_n','N_r','N_f'])